# Chapter 16: Objects and classes: philosophy and iterators.

You will use everything you've learned up to this point in this chapter.

It should be fun!

I'm going to start this chapter by giving you something practical:
an example of a fully-formed class that shows you how classes are used.

A rational number is one of the form *a*/*b*, where *a* and *b* are integers. Python does not have built-in support for rationals and rational arithmetic. 

In [ ]:
class RationalNumber:
    """A class that implements a rational number and the necessary
    Arithmetic operations on it."""
    def __init__(self,numerator, denominator):
        """Arguments should be numbers or RationalNumbers, and will
        be the values of this rational number's numerator and denominator."""
        if(isinstance(numerator, RationalNumber)):
            if(isinstance(denominator, RationalNumber)):
               #The constructor was called with RationalNumbers
               self._n = numerator._n * denominator._d
               self._d = denominator._n * numerator._d
            else:
                #The numerator, but not denominator, is a RationalNumber
                self._n = numerator._n
                self._d = denominator* numerator._d
        else:
            if(isinstance(denominator, RationalNumber)):
               #The denominator, but not numerator, is a RationalNumber
               self._n = numerator * denominator._d
               self._d = denominator._n
            else:
                #Both arguments are plain old numbers
                self._n = numerator
                self._d = denominator
        if(self._n != 0):
            self.reduceFraction()
        else:
            self._d = 1

    def reduceFraction(self):
        gcd = greatestDivisor(self._n, self._d)
        self._n //= gcd
        self._d //= gcd

    def add(self, otherNum):
        """Adds a rational number to this one, using the fact that
        a/b + c/d = (a*d + c*b)/(b*d)"""
        return RationalNumber(self._n*otherNum._d + otherNum._n * self._d, self._d * otherNum._d)

    def subtract(self,otherNum):
        negOther = RationalNumber(-otherNum._n, otherNum._d)
        return self.add(negOther)

    def mult(self, otherNum):
        return RationalNumber(self._n * otherNum._n, self._d * otherNum._d)

    def divide(self, otherNum):
        return RationalNumber(self._n * otherNum._d, self._d * otherNum._n)
    def __str__(self):
        return "{0:d}/{1:d}".format(self._n, self._d)
    
#I put the code for GCD outside the class - it's not really associated with rational numbers, 
#so it should be in a different place.

def greatestDivisor(a,b):
    if(b == 0):
        return a
    return greatestDivisor(b,a % b)

def useRational():
    #a = 1/2
    a = RationalNumber(1,2)
    #b = 1/3
    b = RationalNumber(1,3)
    #c = a + b
    c = a.add(b)
    print(c)
    #Now to demonstrate that rationals are truly precise...
    storage = RationalNumber(0,1)
    floatSum = 0
    for i in range(1000):
        storage = storage.add(RationalNumber(1,1000))
        floatSum += 0.001
    print(floatSum)
    print(storage)
    floatZero = floatSum - 1.0
    storageZero = storage.subtract(RationalNumber(1,1))
    print(floatZero)
    print(storageZero)
    #The floating point version has some noise that has accumulated during
    #the computation. The rational does not have this noise.

Next: Something practical. You know how you can do

In [ ]:
#for i in range(10):

, right? Well, *range* is just a class with a few methods defined.

In [ ]:
#A class is iterable (may be used with a for loop) if it defines the method __iter__() that returns 
# an object with a method called __next__(). __next__() should return the next value in the sequence or raise
#a StopIteration exception. 

class NewRange():
    def __init__(self, start, stop):
        print("NewRange.__init__")
        self._start = start
        self._stop = stop
    def __iter__(self):
        print("NewRange.__iter__")
        return RangeIterator(self._start,self._stop)

class RangeIterator():
    def __init__(self,start,stop):
        print("RangeIterator.__init__")
        self._currPos = start
        self._endPos = stop
    def __next__(self):
        print("RangeIterator.__next__", end = " ")
        if self._currPos < self._endPos:
            self._currPos = self._currPos + 1
            print(" -> {0:d}".format(self._curPos-1))
            return self._currPos - 1 #-1 because I already incremented, return
        else:                        #what the value was, not what it is. 
            print(" -> StopIteration")
            raise StopIteration
    
    
#If your class contains a method called __next__(), you can have __iter__
#just return self:

class SimpleRange:
    def __init__(self,start,stop):
        self._currPos = start
        self._endPos = stop
    def __next__(self):
        if self._currPos < self._endPos:
            self._currPos = self._currPos + 1
            return self._currPos - 1 #-1 because I already incremented, return
        else:                        #what the value was, not what it is. 
            raise StopIteration
    def __iter__(self):
        return self

#When Python comes to a for loop, it first calls __iter__(), then repeatedly
#calls __next__() on that iterator until it throws StopIteration.

#The advantage is we can just use it like a normal range. 
def useNewRange():
    nr = NewRange(0,10)
    for i in nr:
        print (i)
    sr = SimpleRange(0,10)
    for i in sr:
        print(i)

Okay, let's get biochemical again. Consider a class that stores DNA:

In [ ]:
class DNAStore:
    """Represents a strand of DNA. Accepts new dna as strings or collections
    of strings. """
    _bases = "" #Currently empty.

    def __init__(self, bases):
        """bases is a string or a sequence of strings that will be added to
        this objects' dna store."""
        self.add(bases)
        print("Initialized DNA strand with {0:s}".format(self._bases))

    def add(self, newDNA):
        """Adds new dna to the end of this strand. Rules for dna are the same
        as for the initializer."""
        if isinstance(newDNA, str):
            for base in newDNA:
                self._addLetter(base)
        elif isinstance(newDNA, (tuple,list)):
            for thing in newDNA:
                self.add(thing) #If it's a tuple or list, split it and add
                                 #each part of it recursively.
        else:
            raise Exception("Invalid DNA.")
        
    def _addLetter(self, base):
        if base in "AGTC":
            self._bases = self._bases + base
        else:
            raise Exception("Unknown letter for DNA: {0:s}".format(base))

    def getBases(self):
        return self._bases
    

#I'd like to extend this class to allow me to iterate over the codons.

class IterableDNA(DNAStore):
    """An iterable version of a DNA store. Iterates by *codon*, not by
    *base*."""
    _bases = "" #Currently empty.

    def __init__(self, bases):
        """bases is a string or a sequence of strings that will be added to
        this objects' dna store."""
        self.add(bases)
        print("Initialized DNA strand with {0:s}".format(self._bases))

    def add(self, newDNA):
        """Adds new dna to the end of this strand. Rules for dna are the same
        as for the initializer."""
        if isinstance(newDNA, str):
            for base in newDNA:
                self._addLetter(base)
        elif isinstance(newDNA, (tuple,list)):
            for thing in newDNA:
                self.add(thing) #If it's a tuple or list, split it and add
                                 #each part of it recursively.
        else:
            raise Exception("Invalid DNA.")
        
    def _addLetter(self, base):
        if base in "AGTC":
            self._bases = self._bases + base
        else:
            raise Exception("Unknown letter for DNA: {0:s}".format(base))

    def getBases(self):
        return self._bases

    def __iter__(self):
        #Initialize the iteration. 
        self._iterPos = 0
        return self
    def __next__(self):
        start = self._iterPos
        self._iterPos = start + 3
        if(len(self._bases) - start < 3):
            raise StopIteration
        codon = self._bases[start:start + 3]
        return codon

def iterateDNA():
    idna = IterableDNA("AGTGACTAGTCACTACTAGCATGAGACATGACGAT")
    for cdn in idna:
        print(cdn)
        #The big point here is that the person using your class needn't
        #think about how the iteration works; it "just works" and is clear
        #and simple.

####   *Exercises*

1 - Add a method to DNAStore that calculates the GC content of its stored dna.

2 - Add a method to DNAStore that accepts another DNAStore, and calculates the Hamming distance between itself and the other strand.

3 - Explain the behavior of this function:

In [ ]:
def rangeMess():
    def printNest(iterable):
        for i in iterable:
            for j in iterable:
                print("i = {0}, j = {1}.".format(i,j))
    
    a = range(0,10)
    b = NewRange(0,10)
    c = SimpleRange(0,10)
    print("built-in range:")
    printNest(a)
    print("NewRange:")
    printNest(b)
    print("SimpleRange:")
    printNest(c)

4 - If you play with *IterableDNA*, you'll notice it has the behavior of *SimpleRange*: You can't nest iteration. Fix it.

In [ ]:
class BetterIterableDNA:
    pass    

5 - Implement a *deque* class.
(See Chapter 12, *circles*() for a brief discussion of deques.)

It should support these operations:

In [ ]:
#pushLeft(thing) :: appends thing to the left end of the deque.
#popLeft()       :: removes the leftmost item from the deque.
#peekLeft()      :: returns the leftmost item from the deque.

#and their corresponding right-side methods.

class Deque:
    pass

#I have provided this test method for your use:
def testDeque():
    def checkEqual(a,b):
        if (a != b):
            raise Exception("unequal: {0}, {1}".format(a,b))
    def checkBroken(op):
        """Tries to run op (which should be a zero-argument function). If op raises
    an exception, this catches it and returns gracefully. If op does *not* raise
    an exception, this raises its own to indicate that the code did not fail."""
        try:
            op()
        except(Exception):
            print("Error occured as expected.")
            return
        raise Exception("Code did not indicate an error.")
                                    # D1            D2
    d1 = Deque()                    # <>
    d1.pushLeft(1)                  # <1>
    d1.pushRight(2)                 # <1, 2>
    checkEqual(d1.peekLeft(), 1)    # <1, 2>
    checkEqual(d1.peekLeft(), 1)    # <1, 2>
    d1.popLeft()                    # <2>
    checkEqual(d1.peekLeft(), 2)    # <2>
    #Can the class support being emptied?
    d1.popRight()                   # <>
    #Does the class support strange objects being inserted?
    d1.pushRight((3,4))             # <(3,4)>
    d1.pushLeft("aoeu")             # <"aoeu", (3,4)>
    checkEqual(d1.peekRight(), (3,4))#<"aoeu", (3,4)>
    d2 = Deque()                    #  '            <>
    d2.pushLeft(2)                  #  '            <2>
    #Are multiple objects truly independent?
    checkEqual(d2.peekRight(), 2)   #  '            <2>
    d1.popLeft()                    #  <(3,4)>      <2>
    d1.popLeft()                    #  <>           <2>
    #Beat up the class a bit...
    for i in range(10000):
        d1.pushLeft(i)              # <10000, 9999, ... 1, 0>
    for i in range(5000):
        d1.popRight()               #<10000, 9999, ... 5001, 5000>
    checkEqual(d1.peekRight(), 5000)

    d3 = Deque()
    #Does it indicate a problem if I try to remove or read from an empty deque?
    checkBroken(lambda:d3.popRight())
    checkBroken(lambda:d3.peekLeft())
    #Does the deque still work correctly after I try to manipulate it when
    #empty?
    d3.pushLeft(1)
    checkEqual(d3.peekRight(),1)

6 - Make your deque class iterable. The iteration should start at the left and yield all the elements, just like for a list. 

Iterating should NOT destroy the deque being used. That is, after I iterate it, I should be able to push
and pop and peek just as before and all the values must be the same.

In [ ]:
# As an example, the following __next__() would violate this requirement:
#def __next__(self):
#    if(self._isEmpty()):
#         raise StopIteration
#    self.popLeft()
#    return self.peekLeft()
#(Assuming, of course, that self refers to the original deque)

(If you implemented your deque well, this should not be hard!) Note: You may assume that the deque is not modified during the iteration, so, for example, the behavior of the following code is undefined, and will not be tested:

In [ ]:
#   for elem in deq:
#       deq.popRight() #Undefined behavior: Deque is modified during iteration.
#       print(elem)
#       elem = elem+1  #Also undefined: I'm trying to modify the elements.

You can assume that the iterator will not be nested; if it works like *SimpleRange*, that's okay.

In [ ]:
class IterableDeque(Deque):
    pass

7 - Write a method to stress-test your deque, like the tests above.

In [ ]:
def testIterableDeque():
    pass